В этом блокноте представлены результаты распознавания речи на русском языке в аудиозаписи, в которой присутствует сильное эхо и посторонние шумы.

Распознавание выполнено несколькими реализациями Whisper ASR, чтобы сравнить результаты и понять основные трудности, которые возникают при распознавании аудиозаписей с сильным эхом и посторонними шумами.

В идеале необходимо дообучить Whisper ASR на аудиозаписях с шумами, эхом и речью на русском языке. Но на данный момент я не знаю способов дообучения Whisper ASR. В теории дообучить для более точного распознавания речи (на русском языке) возможно, так как Whisper ASR хорошо распознаёт речь на английском языке в присутствии шумов и эха.

В дальнейшем планирую найти audio enhancer, который будет улучшать качество речи в аудиозаписи, удалять эхо и посторонние шумы. Это должно повысить качество/точность распознавания речи на русском языке, так как Whisper ASR отлично распознаёт речь в аудиозаписях без шумов и эха.

Перед запуском в Google Colab необходимо сменить среду выполнения (если она уже не выбрана по-умолчанию):

Runtime -> Change runtime type -> Hardware accelerator -> T4 GPU

In [1]:
# загрузка аудиофайла с Google-диска

import requests
import os

current_dir = os.path.abspath('')
parent_dir = os.path.dirname(current_dir)


def download_file_from_cloud_storage(id, destination):
    url = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    params = { 'id': id, 'confirm': 1}
    response = session.get(url, params = params, stream = True)

    save_response_content(response, destination)


def save_response_content(response, destination):
    chunk_size = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(chunk_size):
            if chunk:
                f.write(chunk)


download_file_from_cloud_storage('17u5p6LI-bo8wrWV3fF1adDM24EOFU-Zn',
                                 os.path.join(parent_dir, 'audio.wav'))

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.5 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.2 MB/s eta 0:00:00


In [4]:
import datetime

# Hugging Face FP32

In [5]:
import torch
from transformers import pipeline

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
pipe = pipeline(
    'automatic-speech-recognition',
    model='openai/whisper-large-v2',
    chunk_length_s=30,
    device=device,
    generate_kwargs={'task':'transcribe'}
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
t1 = datetime.datetime.now()
print(f"Start: {t1}")
prediction = pipe(os.path.join(parent_dir, 'audio.wav'),
                  batch_size=8,
                  return_timestamps=True)
t2 = datetime.datetime.now()
print(f"End: {t2}")
print(f"Elapsed: {t2 - t1}")

Start: 2023-10-13 10:24:04.921066
End: 2023-10-13 10:27:36.906720
Elapsed: 0:03:31.985654


Требования для запуска этой версии.

Оперативная память системы: 9 GB.

Оперативная память графичесого процессора: 12.5 GB.

In [7]:
prediction['text']

' Я стараюсь говорить хорошо по заправлению. Итак, послезавтра у вас начинается период сдачи вступительного рецепта. Мы нас проводим. Я понимаю, что это работа уже встретили по специальности. Сегодня же у меня активно проходят. Но тем, но тем не менее, на самом деле это наше всё. В течение всего года мы тоже будем с вами. Я надеюсь, практически со всеми, кто здесь присутствует, это знак. Будем встречаться на занятиях. Но перед этим вам предстоит вспомнить, как оно звучит. Я думаю, что все из вас читали как раз. Ничего сложного в ней нет. Но тем не менее, я просто вам предварю основные задания, которые вам предстоит выполнить, чтобы получить максимально положительную оценку в стабилу. У нас специально разбитых Вы читали, что тут происходит, как это все ценивается Проговорим еще раз, просто чтобы снять вопросы Что, как, где, зачем, почему Итак, первый вопрос Это ваша траектория навыка работы с общей научной тематикой Как почерк Это не специализированном тексте. Там не будет никакой специ

# OpenAI Original Large FP16

In [3]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print('GPU')
else:
  device = torch.device("cpu")
  print("CPU")

GPU


In [4]:
import whisper
import datetime
import os
model = whisper.load_model('large-v2')
t1 = datetime.datetime.now()
print(f"Start: {t1}")
output = model.transcribe(os.path.join(parent_dir, 'audio.wav'))
t2 = datetime.datetime.now()
print(f"End: {t2}")
print(f"Elapsed: {t2 - t1}")

100%|█████████████████████████████████████| 2.87G/2.87G [00:39<00:00, 77.7MiB/s]


Start: 2023-10-13 10:37:24.569553
End: 2023-10-13 10:45:39.217497
Elapsed: 0:08:14.647944


Требования для запуска этой версии.

Оперативная память системы: 8.6 GB.

Оперативная память графичесого процессора: 10.6 GB.

In [5]:
output['text']

" После завтра вас начинает соблюдать задачи вступительного экзамена и на странном узлу. Я понимаю, что для вас это уже встретили на специальности сегодня, вернее, активно проходят, но тем не менее, на странном узле наше все. И в течение всего года мы тоже будем с вами. Я надеюсь, практически со всеми, кто здесь присутствует, это знак. Будем встречаться на занятиях. Но перед этим вам предстоит немножко вспомнить, как она звучит. Я думаю, что все из вас читали как раз. Ничего сложного в ней нет, но тем не менее, я просто вам предварительно говорю, основные задания, которые вам предстоит выполнить, для того, чтобы получить максимально положительную оценку в ступанах, у нас специально раздельных. Вы читали, что тут расходится, как это все ценивается. Поговорим еще раз, просто чтобы снять вопросы, что, как, где, зачем, почему. Итак, первый вопрос. Это ваша проекта. Нам это работает с общим научным научным учебным. Как почем? Вот это не специализированные теории. Никакой специфики терминоло

# Faster Whisper Large FP16

In [2]:
!pip install faster-whisper==0.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.3 MB/s eta 0:00:00


In [3]:
from faster_whisper import WhisperModel
import datetime
import os
model_size = "large-v2"
model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe(os.path.join(parent_dir, 'audio.wav'),
                                  beam_size=5)
print("Detected language '%s' with "
      "probability %f" % (info.language, info.language_probability))

output_text = ''
t1 = datetime.datetime.now()
print(f"Start: {t1}")
for segment in segments:
    output_text += segment.text
t2 = datetime.datetime.now()
print(f"End: {t2}")
print(f"Elapsed: {t2 - t1}")

(…)32dc1fbce16f98df699208837e8b/config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

(…)c1fbce16f98df699208837e8b/tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

(…)c1fbce16f98df699208837e8b/vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Detected language 'ru' with probability 0.692383
Start: 2023-10-13 11:36:57.240928
End: 2023-10-13 11:40:47.874672
Elapsed: 0:03:50.633744


Требования для запуска этой версии.

Оперативная память системы: 3.8 GB.

Оперативная память графичесого процессора: 5.3 GB.

In [4]:
output_text

' Я стараюсь говорить хорошо по-фамилийному, давайте с ним поучимся. Итак, я хочу сказать, что у меня есть много проблем, я не могу объяснить все, что происходит, я не могу объяснить все, что происходит, я не могу объяснить все, что происходило. Поэтому послезавтра будет сделана выступительнаяuce настраиваемаяuce. Я пускаюаб его, как профессионалю, но Everyday stalker я буду дис tax Но перед этим вам предстоит немного вспомнить, как оно звучит. Я думаю, что все из вас читали как раз. Ничего сложного в ней нет. Но тем не менее, я просто предварю основные задания, которые вам предстоит выполнить, чтобы получить максимально положительную оценку в 100 баллов. У нас специальный разбиратель. Вы читали, что туда входит, как это все оценивается. Поговорим еще раз просто, чтобы снять вопросы. Что, как, где, зачем, почему. Итак, первый вопрос. Это ваша проекта. Навык работы с общей научной информацией. Как почему? Вот это не специализированное дело. Там не будет никакой специфики терминологическ

# Faster Whisper Large INT8_FP16

In [2]:
!pip install faster-whisper==0.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.1 MB/s eta 0:00:00


In [3]:
from faster_whisper import WhisperModel
import datetime
import os
model_size = "large-v2"
model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
segments, info = model.transcribe(os.path.join(parent_dir, 'audio.wav'),
                                  beam_size=5)
print("Detected language '%s' with "
      "probability %f" % (info.language, info.language_probability))

output_text = ''
t1 = datetime.datetime.now()
print(f"Start: {t1}")
for segment in segments:
    output_text += segment.text
t2 = datetime.datetime.now()
print(f"End: {t2}")
print(f"Elapsed: {t2 - t1}")

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

(…)c1fbce16f98df699208837e8b/tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

(…)c1fbce16f98df699208837e8b/vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

(…)32dc1fbce16f98df699208837e8b/config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

Detected language 'ru' with probability 0.680176
Start: 2023-10-13 11:48:09.342940
End: 2023-10-13 11:51:15.866389
Elapsed: 0:03:06.523449


Требования для запуска этой версии.

Оперативная память системы: 4.2 GB.

Оперативная память графичесого процессора: 3.6 GB.

In [4]:
output_text

' Послезавтра у вас начинается период сдачи вступительного экземпляра и настроенного вызова Я понимаю, что это работа уже встретили на специальности сегодня, вернее, активно проходит Но, тем не менее, настроенный экземпляр — это наше всё И в течение всего года мы тоже будем с вами Я надеюсь, практически со всеми, кто здесь присутствует, это знак Будем встречаться на занятиях И перед этим вам предстоит вспомнить, как она звучит Я думаю, что все из вас читали как раз Ничего сложного в ней нет Но, тем не менее, я просто предварю основные задания, которые вам предстоит выполнить Чтобы получить максимально положительную оценку в 100 баллов У нас специальный раздельник Вы читали, что тут у нас ходит, как это всё оценивается Поговорим еще раз просто, чтобы снять вопросы, что, как, где, зачем и почему Итак, первый вопрос — это ваша коллега Нам это работает с общей научной тематикой Это не специализированные тексты Там нет никакой специфики терминологически привязанной к какой-то научной специа

# Faster Whisper Large INT8

In [2]:
!pip install faster-whisper==0.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 116.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.0 MB/s eta 0:00:00


In [3]:
from faster_whisper import WhisperModel
import datetime
import os
model_size = "large-v2"
model = WhisperModel(model_size, device="cuda", compute_type="int8")
segments, info = model.transcribe(os.path.join(parent_dir, 'audio.wav'),
                                  beam_size=5)
print("Detected language '%s' with "
      "probability %f" % (info.language, info.language_probability))

output_text = ''
t1 = datetime.datetime.now()
print(f"Start: {t1}")
for segment in segments:
    output_text += segment.text
t2 = datetime.datetime.now()
print(f"End: {t2}")
print(f"Elapsed: {t2 - t1}")

(…)32dc1fbce16f98df699208837e8b/config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

(…)c1fbce16f98df699208837e8b/vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

(…)c1fbce16f98df699208837e8b/tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Detected language 'ru' with probability 0.680176
Start: 2023-10-13 11:55:51.862152
End: 2023-10-13 11:59:03.988760
Elapsed: 0:03:12.126608


Требования для запуска этой версии.

Оперативная память системы: 4.3 GB.

Оперативная память графичесого процессора: 2.8 GB.

In [4]:
output_text

' Послезавтра у вас начинается период сдачи вступительного экземпляра и настроенного вызова Я понимаю, что это работа уже встретили на специальности сегодня, вернее, активно проходит Но, тем не менее, настроенный экземпляр — это наше всё И в течение всего года мы тоже будем с вами Я надеюсь, практически со всеми, кто здесь присутствует, это знак Будем встречаться на занятиях И перед этим вам предстоит вспомнить, как она звучит Я думаю, что все из вас читали как раз Ничего сложного в ней нет Но, тем не менее, я просто предварю основные задания, которые вам предстоит выполнить Чтобы получить максимально положительную оценку в 100 баллов У нас специальный раздельник Вы читали, что тут у нас ходит, как это всё оценивается Поговорим еще раз просто, чтобы снять вопросы, что, как, где, зачем и почему Итак, первый вопрос — это ваша коллега Нам это работает с общей научной тематикой Это не специализированные тексты Там нет никакой специфики терминологически привязанной к какой-то научной специа